<!-- Team Banner -->
<div align="center">
  <h1>🌟<strong>EVER LEARNERS</strong>🌟</h1>
</div>


<!-- Team Banner -->
<div align="center">
  <h2><strong>TRACK 5 CODE FOR MPEG DIALOGUE COMPETITION– OPEN-ENDED DISCOVERY (INTEGRATED DECODING PIPELINE AND MULTIMODAL HEALTH CLASSIFICATION)</strong></h2>
</div>

<div align="center">
  <img src="https://zindi-public-release.s3.eu-west-2.amazonaws.com/uploads/competition/image/591/thumb_f7f99618-bc3d-44db-acfa-39b971d7472f.png" alt="Description" style="width:50%;"/>
</div>


In [ ]:
%%capture
!pip install pyfiglet

In [1]:
# deterministic_eva_training.py
import os
# must be first!
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"  
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_cytokines_df = pd.read_csv('/kaggle/input/mpeg-g-dialogue/cytokine_profiles.csv')
train_taxa_df = pd.read_csv('/kaggle/input/taxa-new-data-microbiome/kraken_results_all_files.csv')
train_df = pd.read_csv('/kaggle/input/mpeg-g-dialogue/Train.csv')
train_subjects = pd.read_csv('/kaggle/input/mpeg-g-dialogue/Train_Subjects.csv')

In [3]:
train_taxa_df

,filename,taxa,reads_clade,reads_taxon,rank
0,ID_AAFNOT,unclassified,1318,1318,U
1,ID_AAFNOT,root,41522,0,R
2,ID_AAFNOT,cellular organisms,41522,0,R1
3,ID_AAFNOT,Bacteria,41522,64,D
4,ID_AAFNOT,Terrabacteria group,38198,13,D1
...,...,...,...,...,...
954436,ID_ZZXNUV,Desulfurobacteriales,1,0,O
954437,ID_ZZXNUV,Desulfurobacteriaceae,1,0,F
954438,ID_ZZXNUV,Thermovibrio,1,0,G
954439,ID_ZZXNUV,Thermovibrio ammonificans,1,0,S


In [4]:
display(train_df, train_subjects)

,filename,SampleType,SubjectID,SampleID
0,ID_LETPJN.mgb,Stool,Subject_BCUNIB,Sample_AFTIWE
1,ID_NTDGIW.mgb,Stool,Subject_UDAXIH,Sample_JQJVNK
2,ID_ZISBMF.mgb,Nasal,Subject_VRKADI,Sample_YJWGWW
3,ID_MDCVXU.mgb,Nasal,Subject_HIFZWD,Sample_VKEHZP
4,ID_SWAMNY.mgb,Stool,Subject_VGLNCV,Sample_NTDTEQ
...,...,...,...,...
2896,ID_YBMPZU.mgb,Mouth,Subject_NHOSIZ,Sample_RAZQOY
2897,ID_KIVZDA.mgb,Mouth,Subject_OZRDBU,Sample_GKJHPV
2898,ID_IDGAEG.mgb,Skin,Subject_IBWNAQ,Sample_GHULTX
2899,ID_TGSMCA.mgb,Skin,Subject_MBPHGO,Sample_DBDVLO


,SubjectID,FPG_Mean,FPG_class,IRIS,SSPG,FPG,SSPG.Date,Class,Gender,Ethnicity,Adj.age,BMI,OGTT,OGTT_Class,Longitudinal.HbA1C.Group,A1C_Class,Family
0,Subject_UDAXIH,1.274432,Diabetes,IS,91.5,131.75,8/7/14,Diabetic,M,C,59.48,21.47,2.245000,Diabetes,6. Variable Diabetic-PreDM (n = 8),6.VDP,NaN
1,Subject_NHOSIZ,0.915833,Normal,Unknown,NaN,NaN,NaN,Prediabetic,M,C,61.17,27.06,1.005000,Normal,3. PreDM-to-Normal (n = 10),3.PN,NaN
2,Subject_AYZFWN,0.952000,Normal,Unknown,NaN,NaN,NaN,Prediabetic,F,A,56.22,30.61,NaN,NaN,1. Normal (n = 51),1.Normal,NaN
3,Subject_KIEGYK,1.160000,Prediabetes,Unknown,NaN,NaN,NaN,Prediabetic,M,C,49.18,23.22,NaN,NaN,1. Normal (n = 51),1.Normal,NaN
4,Subject_CBVHYJ,0.880000,Normal,Unknown,NaN,NaN,NaN,Control,F,C,46.04,25.26,0.930000,Normal,1. Normal (n = 51),1.Normal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Subject_FSIBDT,0.845909,Normal,IR,174.0,94.00,7/8/13,Prediabetic,F,C,61.95,30.59,1.455000,Prediabetes,4. Variable Normal-PreDM (n = 21),4.VNP,NaN
62,Subject_TQDMSG,0.834615,Normal,IS,40.0,87.00,4/23/13,Prediabetic,F,C,50.02,26.94,1.125000,Normal,1. Normal (n = 51),1.Normal,NaN
63,Subject_IBWNAQ,1.038571,Prediabetes,IR,234.0,115.00,5/7/13,Crossover,F,A,57.72,28.89,2.380000,Diabetes,5. Prediabetic (PreDM) (n = 14),5.P,NaN
64,Subject_ZRVXXX,0.901364,Normal,IS,75.0,94.00,10/15/13,Prediabetic,F,B,49.69,28.24,1.146667,Normal,1. Normal (n = 51),1.Normal,NaN


In [5]:
train_df = pd.merge(train_df, train_subjects, on='SubjectID')

In [6]:
train_df['filename'] = train_df['filename'].str.replace(".mgb", "")

In [7]:
train_df

,filename,SampleType,SubjectID,SampleID,FPG_Mean,FPG_class,IRIS,SSPG,FPG,SSPG.Date,Class,Gender,Ethnicity,Adj.age,BMI,OGTT,OGTT_Class,Longitudinal.HbA1C.Group,A1C_Class,Family
0,ID_LETPJN,Stool,Subject_BCUNIB,Sample_AFTIWE,1.063333,Prediabetes,Unknown,NaN,NaN,NaN,Prediabetic,F,C,41.65,40.83,1.870000,Prediabetes,5. Prediabetic (PreDM) (n = 14),5.P,NaN
1,ID_NTDGIW,Stool,Subject_UDAXIH,Sample_JQJVNK,1.274432,Diabetes,IS,91.5,131.75,8/7/14,Diabetic,M,C,59.48,21.47,2.245000,Diabetes,6. Variable Diabetic-PreDM (n = 8),6.VDP,NaN
2,ID_ZISBMF,Nasal,Subject_VRKADI,Sample_YJWGWW,0.899545,Normal,IR,220.0,95.00,12/18/12,Prediabetic,M,C,62.88,32.94,1.891667,Prediabetes,4. Variable Normal-PreDM (n = 21),4.VNP,NaN
3,ID_MDCVXU,Nasal,Subject_HIFZWD,Sample_VKEHZP,0.800000,Normal,IR,235.0,100.00,5/30/12,Prediabetic,M,C,60.71,34.15,1.510000,Prediabetes,3. PreDM-to-Normal (n = 10),3.PN,NaN
4,ID_SWAMNY,Stool,Subject_VGLNCV,Sample_NTDTEQ,0.882667,Normal,IR,162.0,99.00,10/21/13,Control,F,C,58.65,31.24,1.028333,Normal,1. Normal (n = 51),1.Normal,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2896,ID_YBMPZU,Mouth,Subject_NHOSIZ,Sample_RAZQOY,0.915833,Normal,Unknown,NaN,NaN,NaN,Prediabetic,M,C,61.17,27.06,1.005000,Normal,3. PreDM-to-Normal (n = 10),3.PN,NaN
2897,ID_KIVZDA,Mouth,Subject_OZRDBU,Sample_GKJHPV,0.860000,Normal,Unknown,NaN,NaN,NaN,Prediabetic,M,C,71.56,26.85,0.995000,Normal,4. Variable Normal-PreDM (n = 21),4.VNP,C
2898,ID_IDGAEG,Skin,Subject_IBWNAQ,Sample_GHULTX,1.038571,Prediabetes,IR,234.0,115.00,5/7/13,Crossover,F,A,57.72,28.89,2.380000,Diabetes,5. Prediabetic (PreDM) (n = 14),5.P,NaN
2899,ID_TGSMCA,Skin,Subject_MBPHGO,Sample_DBDVLO,0.827857,Normal,IR,175.0,90.00,6/1/12,Control,M,A,51.13,29.31,0.985000,Normal,1. Normal (n = 51),1.Normal,NaN


In [8]:
train_taxa_df_pivot = train_taxa_df.pivot_table(
    index="filename",
    columns="taxa",
    values="reads_taxon",
    aggfunc="sum",
    fill_value=0
)
train_taxa_df_pivot = train_taxa_df_pivot.reset_index()

In [9]:
train_taxa_df_pivot

taxa,filename,Homo sapiens,Homo,Homininae,Hominidae,Hominoidea,Catarrhini,Simiiformes,Haplorrhini,Primates,...,Thermotogae,unclassified Bacteria,Archaea,Bacteria,Eukaryota,"dsDNA viruses, no RNA stage",Viruses,cellular organisms,root,unclassified
0,ID_AAFNOT,0,0,0,0,0,0,0,0,0,...,0,0,0,64,0,0,0,0,0,1318
1,ID_AAXPTO,0,0,0,0,0,0,0,0,0,...,0,0,0,1553,0,0,0,0,0,7
2,ID_AAYKAN,0,0,0,0,0,0,0,0,0,...,0,0,0,38,0,0,0,0,0,0
3,ID_ABEZNS,0,0,0,0,0,0,0,0,0,...,0,0,0,79,0,0,0,0,0,34478
4,ID_ABFFLP,0,0,0,0,0,0,0,0,0,...,0,0,0,158,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2896,ID_ZZJLKK,0,0,0,0,0,0,0,0,0,...,0,0,0,81,0,0,0,0,0,3980
2897,ID_ZZMKVJ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2898,ID_ZZTOHP,0,0,0,0,0,0,0,0,0,...,0,0,0,33,0,0,0,0,0,0
2899,ID_ZZVNNS,0,0,0,0,0,0,0,0,0,...,0,0,0,2983,0,0,0,0,0,14565


In [10]:
train_cytokines_df

,SampleID,Plate,IL17F,FASL,TGFA,MIP1A,SDF1A,IL27,LIF,IL1B,...,ENA78,CHEX1,CHEX2,CHEX3,CHEX4,CollectionDate,CL1,CL2,CL3,CL4
0,Sample_BDRJDQ,INF_P8,184.444488,13.607622,20.587480,57.739170,180.058623,22.069171,39.093968,21.497781,...,141.073916,10469.596344,858.356296,1461.456488,12.783104,12/9/14,D3,Cox/Echo/Rhino Infection (Moraxella catarrhali...,Infection_Middle,Infection
1,Sample_ESYUZA,INF_P8,138.577340,14.046578,14.086170,53.462194,174.431791,22.988720,37.590354,24.301840,...,152.087121,10165.621464,751.848760,1260.148504,12.783104,1/5/15,D30,NaN,Infection_Recovery_Late,Infection_L
2,Sample_CNKYCP,INF_P8,155.655534,17.558222,17.336825,53.462194,164.712717,24.368043,34.081921,19.628409,...,126.914080,10167.570021,704.628671,1198.473264,12.291446,6/17/15,D1,NaN,Infection_Early,Infection
3,Sample_IOOXEU,INF_P8,176.637314,18.436133,26.005238,59.877657,154.482114,26.666915,37.089150,20.563095,...,104.887670,10042.375239,825.826901,1348.960850,14.258077,6/19/15,D3,NaN,Infection_Middle,Infection
4,Sample_KELXIR,INF_P8,182.492695,17.558222,17.336825,63.085389,152.435993,23.908268,38.091559,21.497781,...,145.793861,10012.659746,692.036647,1163.935130,13.766419,7/1/15,D15,NaN,Infection_Recovery_Early,Infection_L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,Sample_HIFXVJ,INF_P7,117.297060,18.385297,13.949902,64.023724,197.148603,27.619195,36.810582,18.478845,...,298.227910,10195.140518,710.189107,1193.630511,15.466837,10/9/14,D7,flu?,Imz_Late,Imz
666,Sample_KIOUXQ,INF_P7,129.243982,20.056687,15.112394,72.098788,213.750590,24.659996,32.603658,19.505448,...,269.630713,10392.986809,665.742547,1251.204421,10.112932,10/24/14,D22,flu?,Imz_Recovery_Early,Imz_L
667,Sample_GSYEEV,HMP_P7,80.138347,5.849867,9.159027,18.719467,96.153178,8.218182,29.230866,23.676021,...,250.201646,10282.446408,619.773411,1252.475316,16.872913,4/6/15,NaN,NaN,NaN,Healthy
668,Sample_UVFBSM,HMP_P7,80.943757,6.946717,10.568108,22.285080,107.221170,7.427972,34.852186,18.414683,...,295.692855,10534.572541,737.348073,1346.317123,16.872913,6/8/15,NaN,NaN,NaN,Healthy


In [11]:
df_without_cyto = pd.merge(
    train_df,
    train_taxa_df_pivot,
    on="filename",
    how="inner"  # or "left" if you want all rows from train_df
)
df = pd.merge(
    df_without_cyto,
    train_cytokines_df,
    on="SampleID",
    how="inner"  # or "left" if you want all rows from train_df
)

In [12]:
df

,filename,SampleType,SubjectID,SampleID,FPG_Mean,FPG_class,IRIS,SSPG,FPG,SSPG.Date,...,ENA78,CHEX1,CHEX2,CHEX3,CHEX4,CollectionDate,CL1,CL2,CL3,CL4
0,ID_LETPJN,Stool,Subject_BCUNIB,Sample_AFTIWE,1.063333,Prediabetes,Unknown,NaN,NaN,NaN,...,260.803223,10348.221434,615.135794,1145.764405,15.090773,1/28/16,NaN,NaN,NaN,Healthy
1,ID_MDCVXU,Nasal,Subject_HIFZWD,Sample_VKEHZP,0.800000,Normal,IR,235.0,100.00,5/30/12,...,330.895379,10184.682491,660.826952,1176.694165,15.096921,10/24/14,D3,flu,Imz_Middle,Imz
2,ID_SWAMNY,Stool,Subject_VGLNCV,Sample_NTDTEQ,0.882667,Normal,IR,162.0,99.00,10/21/13,...,304.064072,9424.953170,741.731826,1259.429265,14.277080,2/20/15,NaN,NaN,NaN,Healthy
3,ID_QRIANS,Stool,Subject_CBVHYJ,Sample_MBZSNE,0.880000,Normal,Unknown,NaN,NaN,NaN,...,444.791673,10036.526208,433.323426,981.610550,11.260388,10/14/13,NaN,NaN,NaN,Healthy
4,ID_KSEFAE,Nasal,Subject_UDAXIH,Sample_UZKJRT,1.274432,Diabetes,IS,91.5,131.75,8/7/14,...,131.543279,10025.832505,801.843528,1351.679156,9.249605,6/14/16,D1,NaN,Imz_Early,Imz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977,ID_YBMPZU,Mouth,Subject_NHOSIZ,Sample_RAZQOY,0.915833,Normal,Unknown,NaN,NaN,NaN,...,458.515837,10086.018582,519.160657,1188.399252,9.187504,6/17/13,D7,NaN,NaN,Post-Travel
1978,ID_KIVZDA,Mouth,Subject_OZRDBU,Sample_GKJHPV,0.860000,Normal,Unknown,NaN,NaN,NaN,...,76.683000,10232.637880,333.982992,775.935804,16.510289,4/8/16,NaN,NaN,NaN,Healthy
1979,ID_IDGAEG,Skin,Subject_IBWNAQ,Sample_GHULTX,1.038571,Prediabetes,IR,234.0,115.00,5/7/13,...,1730.290607,10570.730165,390.092676,859.590951,18.819787,4/12/16 12:24,NaN,NaN,NaN,Healthy
1980,ID_TGSMCA,Skin,Subject_MBPHGO,Sample_DBDVLO,0.827857,Normal,IR,175.0,90.00,6/1/12,...,279.570700,10781.800377,381.042450,784.859488,13.349530,10/10/16,NaN,NaN,NaN,Healthy


In [13]:
import os
import random
import numpy as np

# ------------------ ENV & seeds ------------------
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

random.seed(seed)
np.random.seed(seed)

# ------------------ Import PyTorch FIRST (before any settings) ------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# ------------------ Other imports ------------------
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import timm
import warnings
warnings.filterwarnings("ignore")

# ------------------ NOW set PyTorch deterministic settings ------------------
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

try:
    torch.backends.cudnn.allow_tf32 = False
    torch.backends.cuda.matmul.allow_tf32 = False
except:
    pass

try:
    torch.use_deterministic_algorithms(True)
except:
    print("Warning: Could not enable deterministic algorithms")

# ------------------ disable timm fused kernels ------------------
try:
    if hasattr(timm.models, "layers"):
        if hasattr(timm.models.layers, "set_jit_fused"):
            timm.models.layers.set_jit_fused(False)
        if hasattr(timm.models.layers, "set_layer_norm_fused"):
            timm.models.layers.set_layer_norm_fused(False)
except:
    pass

# ------------------ Multimodal Dataset ------------------
class MultimodalDataset(Dataset):
    def __init__(self, df, img_dir, sample_type_enc, class_enc, scaler, 
                 taxa_cols, cytokine_cols, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.sample_type_enc = sample_type_enc
        self.class_enc = class_enc
        self.scaler = scaler
        self.taxa_cols = taxa_cols
        self.cytokine_cols = cytokine_cols
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        filename = str(row["filename"])
        file_id = filename
        img_path = os.path.join(self.img_dir, f"{file_id}_viridis.png")
        
        if not os.path.exists(img_path):
            for suffix in ["_gray.png", "_inferno.png", "_magma.png", "_plasma.png", "_cividis.png"]:
                test_path = os.path.join(self.img_dir, f"{file_id}{suffix}")
                if os.path.exists(test_path):
                    img_path = test_path
                    break
        
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        # Get sample type label (for prediction, NOT as input)
        sample_type = int(self.sample_type_enc.transform([str(row["SampleType"])])[0])
        
        # Get taxa data (microbiome composition)
        taxa_data = row[self.taxa_cols].values.astype(np.float32)
        
        # Get cytokine levels
        cytokine_data = row[self.cytokine_cols].values.astype(np.float32)
        
        # Combine and normalize metadata
        metadata = np.concatenate([taxa_data, cytokine_data])
        metadata = self.scaler.transform(metadata.reshape(1, -1))[0].astype(np.float32)
        
        # Get health status label
        class_label = int(self.class_enc.transform([str(row["Class"])])[0])
        
        # Return: image, metadata, sample_type_label, health_status_label
        return img, torch.tensor(metadata), sample_type, class_label

# ------------------ deterministic worker init ------------------
def worker_init_fn(worker_id):
    worker_seed = seed + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

dl_generator = torch.Generator()
dl_generator.manual_seed(seed)

# ------------------ Multimodal Model with Multi-Task Learning ------------------
class MultimodalHealthModel(nn.Module):
    def __init__(self, model_name, num_sample_types, metadata_dim, num_classes, pretrained=True):
        super().__init__()
        
        # Image encoder (for kmer patterns)
        self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,
            global_pool="avg",
            drop_rate=0.0,
            drop_path_rate=0.0,
        )
        img_features_dim = self.backbone.num_features
        
        # Metadata encoder (for taxa + cytokine data)
        # NO sample type embedding - we're predicting it, not using it as input!
        self.metadata_encoder = nn.Sequential(
            nn.Linear(metadata_dim, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.ReLU()
        )
        
        # Shared fusion layers (without sample type input)
        total_features = img_features_dim + 128
        
        self.shared_fusion = nn.Sequential(
            nn.Linear(total_features, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU()
        )
        
        # Task-specific heads
        # Health status classification head
        self.health_head = nn.Linear(256, num_classes)
        
        # Sample type classification head
        self.sample_type_head = nn.Linear(256, num_sample_types)

    def forward(self, img, metadata):
        # Extract kmer pattern features from image
        img_features = self.backbone(img)
        
        # Encode taxa and cytokine metadata
        metadata_features = self.metadata_encoder(metadata)
        
        # Fuse modalities (NO sample type as input!)
        combined = torch.cat([img_features, metadata_features], dim=1)
        
        # Shared representation
        shared_features = self.shared_fusion(combined)
        
        # Task-specific predictions
        health_output = self.health_head(shared_features)
        sample_type_output = self.sample_type_head(shared_features)
        
        return health_output, sample_type_output

# ------------------ Training Loop with Separate Metrics ------------------
def train_model(train_df, val_df, img_dir, sample_type_enc, class_enc, scaler,
                taxa_cols, cytokine_cols, device, num_sample_types, metadata_dim,
                num_classes, model_name="eva02_large_patch14_224", epochs=10):
    print("\n=== Starting Multi-Task Multimodal Training ===")
    print(f"Image features: k-mer patterns")
    print(f"Metadata: {len(taxa_cols)} taxa/metabolic features + {len(cytokine_cols)} cytokine/immune features")
    print(f"Task 1: Health Status Classification ({num_classes} classes)")
    print(f"Task 2: Sample Type Classification ({num_sample_types} types)")
    print(f"Note: Sample type is PREDICTED, not used as input (avoiding circular dependency)")
    
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225))
    ])

    train_ds = MultimodalDataset(train_df, img_dir, sample_type_enc, class_enc, 
                                  scaler, taxa_cols, cytokine_cols, transform)
    val_ds   = MultimodalDataset(val_df, img_dir, sample_type_enc, class_enc,
                                  scaler, taxa_cols, cytokine_cols, transform)

    train_loader = DataLoader(
        train_ds,
        batch_size=16,
        shuffle=True,
        num_workers=0,
        worker_init_fn=worker_init_fn,
        generator=dl_generator,
        pin_memory=False,
        drop_last=True
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=16,
        shuffle=False,
        num_workers=0,
        pin_memory=False,
        drop_last=False
    )

    model = MultimodalHealthModel(model_name, num_sample_types, metadata_dim, 
                                   num_classes, pretrained=True).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    # Two separate loss functions
    health_criterion = nn.CrossEntropyLoss()
    sample_type_criterion = nn.CrossEntropyLoss()

    best_health_acc = 0.0
    
    for epoch in range(epochs):
        # --------------- TRAIN ---------------
        model.train()
        train_health_correct = 0
        train_sample_type_correct = 0
        train_total = 0
        train_health_loss = 0
        train_sample_type_loss = 0
        
        for imgs, metadata, sample_types, class_labels in train_loader:
            imgs = imgs.to(device)
            metadata = metadata.to(device)
            sample_types = sample_types.to(device)
            class_labels = class_labels.to(device)
            
            optimizer.zero_grad()
            
            health_outputs, sample_type_outputs = model(imgs, metadata)
            
            # Calculate both losses
            health_loss = health_criterion(health_outputs, class_labels)
            sample_type_loss = sample_type_criterion(sample_type_outputs, sample_types)
            
            # Combined loss (you can weight these differently)
            total_loss = health_loss + 0.3 * sample_type_loss
            
            total_loss.backward()
            optimizer.step()
            
            # Health status predictions
            _, health_preds = torch.max(health_outputs, 1)
            train_health_correct += (health_preds == class_labels).sum().item()
            
            # Sample type predictions
            _, sample_type_preds = torch.max(sample_type_outputs, 1)
            train_sample_type_correct += (sample_type_preds == sample_types).sum().item()
            
            train_total += class_labels.size(0)
            train_health_loss += health_loss.item()
            train_sample_type_loss += sample_type_loss.item()
        
        train_health_acc = 100 * train_health_correct / train_total
        train_sample_type_acc = 100 * train_sample_type_correct / train_total
        avg_train_health_loss = train_health_loss / len(train_loader)
        avg_train_sample_type_loss = train_sample_type_loss / len(train_loader)
        scheduler.step()

        # --------------- VALIDATION ---------------
        model.eval()
        val_health_correct = 0
        val_sample_type_correct = 0
        val_total = 0
        val_health_loss = 0
        val_sample_type_loss = 0
        
        all_health_preds = []
        all_health_labels = []
        all_sample_type_preds = []
        all_sample_type_labels = []
        
        with torch.no_grad():
            for imgs, metadata, sample_types, class_labels in val_loader:
                imgs = imgs.to(device)
                metadata = metadata.to(device)
                sample_types = sample_types.to(device)
                class_labels = class_labels.to(device)
                
                health_outputs, sample_type_outputs = model(imgs, metadata)
                
                health_loss = health_criterion(health_outputs, class_labels)
                sample_type_loss = sample_type_criterion(sample_type_outputs, sample_types)
                
                # Health status predictions
                _, health_preds = torch.max(health_outputs, 1)
                val_health_correct += (health_preds == class_labels).sum().item()
                all_health_preds.extend(health_preds.cpu().numpy())
                all_health_labels.extend(class_labels.cpu().numpy())
                
                # Sample type predictions
                _, sample_type_preds = torch.max(sample_type_outputs, 1)
                val_sample_type_correct += (sample_type_preds == sample_types).sum().item()
                all_sample_type_preds.extend(sample_type_preds.cpu().numpy())
                all_sample_type_labels.extend(sample_types.cpu().numpy())
                
                val_total += class_labels.size(0)
                val_health_loss += health_loss.item()
                val_sample_type_loss += sample_type_loss.item()

        val_health_acc = 100 * val_health_correct / val_total
        val_sample_type_acc = 100 * val_sample_type_correct / val_total
        avg_val_health_loss = val_health_loss / len(val_loader)
        avg_val_sample_type_loss = val_sample_type_loss / len(val_loader)

        print(f"\nEpoch {epoch+1}/{epochs}:")
        print(f"  HEALTH STATUS:")
        print(f"    Train - Loss: {avg_train_health_loss:.4f} | Acc: {train_health_acc:.2f}%")
        print(f"    Val   - Loss: {avg_val_health_loss:.4f} | Acc: {val_health_acc:.2f}%")
        print(f"  SAMPLE TYPE:")
        print(f"    Train - Loss: {avg_train_sample_type_loss:.4f} | Acc: {train_sample_type_acc:.2f}%")
        print(f"    Val   - Loss: {avg_val_sample_type_loss:.4f} | Acc: {val_sample_type_acc:.2f}%")

        if val_health_acc > best_health_acc:
            best_health_acc = val_health_acc
            torch.save({
                'model_state_dict': model.state_dict(),
                'sample_type_encoder': sample_type_enc,
                'class_encoder': class_enc,
                'scaler': scaler,
                'taxa_cols': taxa_cols,
                'cytokine_cols': cytokine_cols
            }, "multimodal_health_best.pth")
            print(f"  ✓ New best model saved (based on health status accuracy)!")
            
            # Print detailed metrics for best epoch
            print(f"\n  Detailed Health Status Metrics:")
            print(classification_report(all_health_labels, all_health_preds, 
                                       target_names=class_enc.classes_, 
                                       digits=3, zero_division=0))
            
            print(f"  Detailed Sample Type Metrics:")
            print(classification_report(all_sample_type_labels, all_sample_type_preds,
                                       target_names=sample_type_enc.classes_,
                                       digits=3, zero_division=0))

    print(f"\n=== Training Complete ===")
    print(f"Best Health Status Validation Accuracy: {best_health_acc:.2f}%")
    return model

# ------------------ Run Training ------------------
def run_training(train_df, img_dir, test_size=0.2):
    train_df = train_df.copy()
    
    exclude_cols = ['filename', 'SampleType', 'SubjectID', 'SampleID', 'Class', 
                    'CollectionDate', 'SSPG.Date']
    
    numeric_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
    
    cytokine_patterns = ['IL', 'TNF', 'IFN', 'ENA', 'CHEX', 'CL1', 'CL2', 'CL3', 'CL4']
    cytokine_cols = [col for col in numeric_cols if any(x in col.upper() for x in cytokine_patterns)]
    taxa_cols = [col for col in numeric_cols if col not in cytokine_cols and col not in exclude_cols]
    
    taxa_cols = [col for col in taxa_cols if train_df[col].notna().sum() / len(train_df) > 0.5]
    cytokine_cols = [col for col in cytokine_cols if train_df[col].notna().sum() / len(train_df) > 0.5]
    
    for col in taxa_cols + cytokine_cols:
        train_df[col] = train_df[col].fillna(train_df[col].mean())
    
    print(f"\nFound {len(taxa_cols)} taxa/metabolic features")
    print(f"Found {len(cytokine_cols)} cytokine/immune markers")
    print(f"Total metadata features: {len(taxa_cols) + len(cytokine_cols)}")
    
    sample_type_enc = LabelEncoder()
    class_enc = LabelEncoder()
    
    train_df["sample_type_encoded"] = sample_type_enc.fit_transform(train_df["SampleType"])
    train_df["class_encoded"] = class_enc.fit_transform(train_df["Class"])
    
    metadata_cols = taxa_cols + cytokine_cols
    scaler = StandardScaler()
    scaler.fit(train_df[metadata_cols])

    train_data, val_data = train_test_split(
        train_df, 
        test_size=test_size, 
        stratify=train_df["class_encoded"],
        random_state=seed
    )
    
    print(f"\nTrain samples: {len(train_data)}, Validation samples: {len(val_data)}")
    print(f"Sample Types: {list(sample_type_enc.classes_)}")
    print(f"Health Status Classes: {list(class_enc.classes_)}")

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}\n")

    model = train_model(
        train_data, val_data, img_dir, 
        sample_type_enc, class_enc, scaler,
        taxa_cols, cytokine_cols, device,
        num_sample_types=len(sample_type_enc.classes_),
        metadata_dim=len(metadata_cols),
        num_classes=len(class_enc.classes_)
    )
    return model, sample_type_enc, class_enc

# ------------------ Execute ------------------
model, sample_type_encoder, class_encoder = run_training(
    df, 
    "/kaggle/input/multi-image-mpeg-dataset/5mer_images_parallel"
)


Found 4248 taxa/metabolic features
Found 825 cytokine/immune markers
Total metadata features: 5073

Train samples: 1585, Validation samples: 397
Sample Types: ['Mouth', 'Nasal', 'Skin', 'Stool']
Health Status Classes: ['Control', 'Crossover', 'Diabetic', 'Prediabetic']
Using device: cuda


=== Starting Multi-Task Multimodal Training ===
Image features: k-mer patterns
Metadata: 4248 taxa/metabolic features + 825 cytokine/immune features
Task 1: Health Status Classification (4 classes)
Task 2: Sample Type Classification (4 types)
Note: Sample type is PREDICTED, not used as input (avoiding circular dependency)


model.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]


Epoch 1/10:
  HEALTH STATUS:
    Train - Loss: 1.0142 | Acc: 65.72%
    Val   - Loss: 0.9540 | Acc: 66.25%
  SAMPLE TYPE:
    Train - Loss: 1.3534 | Acc: 33.96%
    Val   - Loss: 1.2540 | Acc: 41.31%
  ✓ New best model saved (based on health status accuracy)!

  Detailed Health Status Metrics:
              precision    recall  f1-score   support

     Control      0.000     0.000     0.000        39
   Crossover      0.000     0.000     0.000        39
    Diabetic      0.000     0.000     0.000        56
 Prediabetic      0.662     1.000     0.797       263

    accuracy                          0.662       397
   macro avg      0.166     0.250     0.199       397
weighted avg      0.439     0.662     0.528       397

  Detailed Sample Type Metrics:
              precision    recall  f1-score   support

       Mouth      0.657     0.742     0.697        93
       Nasal      0.325     0.990     0.490        96
        Skin      0.000     0.000     0.000        93
       Stool      0.

In [1]:
import pyfiglet
from rich.console import Console

console = Console()
# Banner
banner = pyfiglet.figlet_format("EVER LEARNERS", font="slant")
console.print(banner, style="bold cyan")

    _______    ____________ 
   / ____/ |  / / ____/ __ \
  / __/  | | / / __/ / /_/ /
 / /___  | |/ / /___/ _, _/ 
/_____/  |___/_____/_/ |_|  
                            
    __    _________    ____  _   ____________  _____
   / /   / ____/   |  / __ \/ | / / ____/ __ \/ ___/
  / /   / __/ / /| | / /_/ /  |/ / __/ / /_/ /\__ \ 
 / /___/ /___/ ___ |/ _, _/ /|  / /___/ _, _/___/ / 
/_____/_____/_/  |_/_/ |_/_/ |_/_____/_/ |_|/____/  
                                                    